# Downloading Attachments from Gmail

## Imports

In [4]:
import pandas as pd
import datetime
import os
import glob
import email
import imaplib

## Constants

In [5]:
# Login Credentials. Note: Password needs to be set in gmail first through App Password

EMAIL_UN = 'yourmail@gmail.com'
EMAIL_PW = 'xxxxxxxxxxxxxxxx'

subject = '"Credit Card Statement"'

# Constants

month = {
        'jan': '01',
        'feb': '02',
        'mar': '03',
        'apr': '04',
        'may': '05',
        'jun': '06',
        'jul': '07',
        'aug': '08',
        'sep': '09',
        'oct': '10',
        'nov': '11',
        'dec': '12'
        }

## Definitions

In [6]:
def signin_gmail(username, password):
    """
    Connecting to gmail and logging in
    """
    # Gmail IMAP url
    imap_url = 'imap.gmail.com'

    # connecting to the Gmail IMAP server
    conn = imaplib.IMAP4_SSL(imap_url)
    conn.login(username,password)


    # Selecting the Folder for searching for mails
    conn.select('"[Gmail]/All Mail"')
    
    return conn

def download_attachment(file_name, file_content):
    """
    Writes contents of attachment into a file in current directory
    """
    # saving files to current directory
    att_path = os.path.join('.', file_name)
    print(att_path)
    with open(att_path, 'wb') as fp:
        fp.write(file_content)
    
    print(str(file_name)+ ' downloaded')

## Attachment download

In [3]:
m = signin_gmail(EMAIL_UN, EMAIL_PW)

# defining search criteria
resp, items_org = m.search(None, 'SUBJECT', subject)

# splitting all the mail ids to a list
items = items_org[0].split()

for emailid in items:
    print(emailid)
    
    # fetching the mail, "`(RFC822)`" means "get the whole stuff", but you can ask for headers only, etc
    resp, data = m.fetch(emailid, "(RFC822)")
    
    # getting the mail content
    email_body = data[0][1]
    
    # parsing the mail content to get a mail object
    mail = email.message_from_string(str(email_body.decode("utf-8"))) 

    # Check if any attachments at all
    if mail.get_content_maintype() != 'multipart':
        continue

    print("["+mail["From"]+"] :" + mail["Subject"])
    
    name_append = mail["Subject"].split()[::-1][:2]
    name_append[1] = "-{} ".format(month.get(name_append[1][:3].lower())) + name_append[1]
    month_year = "".join(name_append)
    
    intra_mail_counter = 1
    
    # we use walk to create a generator so we can iterate on the parts and forget about the recursive headach
    for part in mail.walk():

        # multipart are just containers, so we skip them
        if part.get_content_maintype() != 'multipart' and part.get('Content-Disposition') is not None:
            filename = part.get_filename()
            filename = filename.split('.')[0] + ' ({}).'.format(month_year) + filename.split('.')[1]
            
            # saving files to current directory
            att_path = os.path.join('.', filename)
            
            # if a file with same name exists, add (counter) to it
            if os.path.isfile(att_path):
                filename = filename.split('.')[0] + ' ({})({}).'.format(month_year, 
                                                                        intra_mail_counter) + filename.split('.')[1]
                
                
            content = part.get_payload(decode=True)
            download_attachment(filename, content)
            
            # if a file with same name exists in the same email, add counter
            intra_mail_counter += 1
    